In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install tensorflow

In [ ]:
# !pip install keras

In [ ]:
import random

import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import cv2


import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract,ZeroPadding2D,Dense, Add
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import random

import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import cv2


import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract,ZeroPadding2D,Dense, Add
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# train, test and validation directories
raw_train_dir = "D:/SHREYASH_CODE/Raw_OCT_Train"
clean_train_dir = "D:/SHREYASH_CODE/Clean_OCT_Train"
raw_val_dir = "D:/SHREYASH_CODE/RawOCT_Val"
clean_val_dir = "D:/SHREYASH_CODE/CleanOCT_Val"
raw_test_dir = "D:/SHREYASH_CODE/RawOCT_Test"
clean_test_dir = "D:/SHREYASH_CODE/CleanOCT_Test"

In [ ]:
walk_through_dir(raw_train_dir)
walk_through_dir(clean_train_dir)
walk_through_dir(raw_val_dir)
walk_through_dir(clean_val_dir)
walk_through_dir(raw_test_dir)
walk_through_dir(clean_test_dir)

In [ ]:
img_folder=raw_train_dir
for i in range(3):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,3,i+1)
    ax.title.set_text(file)
    plt.imshow(img,cmap='gray')
    plt.axis('off')

In [ ]:
img_folder=clean_train_dir
for i in range(3):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,3,i+1)
    ax.title.set_text(file)
    plt.imshow(img,cmap='gray')
    plt.axis('off')

In [ ]:
import cv2
from sklearn.feature_extraction import image
from PIL import Image

def create_dataset(img_folder):
    img_data_array=[]
    path, dirs, files = next(os.walk(img_folder))
    file_count = len(files)
    file_names = sorted(os.listdir(img_folder))
    print(file_count)
    for i in range(file_count):
        imgPath = img_folder+'/'+file_names[i]
        img=Image.open(imgPath)
        img=img.resize((128, 128),Image.LANCZOS)
        img=np.array(img)
        img = img.astype('float32')
        img /= 255
        img_data_array.append(img)
    return img_data_array# extract the image array and class name

In [ ]:
raw_train_data =create_dataset(raw_train_dir)
clean_train_data=create_dataset(clean_train_dir)
raw_test_data=create_dataset(raw_test_dir)
clean_test_data=create_dataset(clean_test_dir)
raw_val_data=create_dataset(raw_val_dir)
clean_val_data=create_dataset(clean_val_dir)

In [ ]:
raw_train_data = np.array(raw_train_data)
clean_train_data = np.array(clean_train_data)
raw_test_data = np.array(raw_test_data)
clean_test_data = np.array(clean_test_data)
raw_val_data = np.array(raw_val_data)
clean_val_data = np.array(clean_val_data)

In [ ]:
raw_train_data.shape

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# defining a function to view image at a particular index
def view_img_at_index(dataset, index):
    plt.figure(figsize=(4, 6))
    plt.imshow(dataset[index], cmap='gray')
    plt.title(f"Image at index {index}: ")
    plt.axis('off')

In [ ]:
import random
i = random.randint(0, (len(raw_train_data)-1))
i

In [ ]:
view_img_at_index(raw_train_data, i)

In [ ]:
view_img_at_index(clean_train_data, i)

In [ ]:
# Custom Convolutional Block
class Cust_Conv(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3,3)):
        super(Cust_Conv, self).__init__()

        self.conv = Conv2D(filters, kernel_size, padding='same')
        self.bn = BatchNormalization()

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)

        return Activation('relu')(x)

In [ ]:
# Residual Blocks
class res_layer(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=(3,3)):
        super(res_layer, self).__init__()

        self.conv1 = Conv2D(filters, kernel_size, padding='same')
        self.bn1 = BatchNormalization()
        self.rel1 = Activation('relu')

        self.conv2 = Conv2D(filters, kernel_size, padding='same')
        self.bn2 = BatchNormalization()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.rel1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        x += inputs
        return Activation('relu')(x)

In [ ]:
# Tensorflow Wavelets

import tensorflow_wavelets.Layers.DWT as DWT
import tensorflow_wavelets.Layers.DMWT as DMWT
import tensorflow_wavelets.Layers.DTCWT as DTCWT
# from keras.models import Sequential
# from keras.layers import Conv2D
# from keras.layers import BatchNormalization
# from keras.layers import Activation

In [ ]:
# DTCWT Layer
from tensorflow.keras import layers
from tensorflow_wavelets.utils import filters
from tensorflow_wavelets.utils.helpers import *
from tensorflow_wavelets.utils.cast import *
class DTCWT(layers.Layer):
    """
    Durel Tree Complex Wavelet Transform
    Input: level - tree-level (int)
    """
    def __init__(self, level=1, **kwargs):
        super(DTCWT, self).__init__(**kwargs)

        if level <= 1:
            level = 1

        self.level = int(level)
        self.conv_type = "SAME"
        self.border_padd = "SYMMETRIC"

        # Faf - First analysis filter - for the first level
        # Fsf - First synthesis filter
        faf, fsf = filters.fs_farras()
        af, sf = filters.duelfilt()

        # convert to tensor
        self.Faf = duel_filter_tf(faf)
        self.Fsf = duel_filter_tf(fsf)
        self.af = duel_filter_tf(af)
        self.sf = duel_filter_tf(sf)

    def build(self, input_shape):
        # repeat last channel if input channel bigger then 1
        if input_shape[-1] > 1:
            self.Faf = tf.repeat(self.Faf, input_shape[-1], axis=-1)
            self.Fsf = tf.repeat(self.Fsf, input_shape[-1], axis=-1)
            self.af = tf.repeat(self.af, input_shape[-1], axis=-1)
            self.sf = tf.repeat(self.sf, input_shape[-1], axis=-1)

    def call(self, inputs, training=None, mask=None):

        # normalize
        x_norm = tf.math.divide(inputs, 2)

        # 2 trees J+1 lists
        w = [[[[], []] for _ in range(2)] for __ in range(self.level+1)]

        # 2 trees - 2 filters ( first stage is using differnet filter
        for m in range(2):
            for n in range(2):
                [lo, w[0][m][n]] = analysis_filter_bank2d(x_norm, self.Faf[m][0], self.Faf[m][1],
                                                          self.Faf[n][0], self.Faf[n][1])
                for j in range(1, self.level):
                    [lo, w[j][m][n]] = analysis_filter_bank2d(lo, self.af[m][0], self.af[m][1],
                                                              self.af[n][0], self.af[n][1])
                w[self.level][m][n] = lo

        # add and subtract for the complex
        for j in range(self.level):
            for m in range(3):

                w[j][0][0][m], w[j][1][1][m] = add_sub(w[j][0][0][m], w[j][1][1][m])
                w[j][0][1][m], w[j][1][0][m] = add_sub(w[j][0][1][m], w[j][1][0][m])

        # concat into one big image
        # different resolution as the tree is deeper
        # TODO: How to split different resolutions into different channels
        j = 1
        w_c = w

        for j in [x for x in range(1, self.level)][::-1]:

            w_c[j][0][0] = tf.concat([tf.concat([w_c[j+1][0][0], w_c[j][0][0][0]], axis=2),
                                      tf.concat([w_c[j][0][0][1], w_c[j][0][0][2]], axis=2)], axis=1)
            w_c[j][0][1] = tf.concat([tf.concat([w_c[j+1][0][1], w_c[j][0][1][0]], axis=2),
                                      tf.concat([w_c[j][0][1][1], w_c[j][0][1][2]], axis=2)], axis=1)
            w_c[j][1][0] = tf.concat([tf.concat([w_c[j+1][1][0], w_c[j][1][0][0]], axis=2),
                                      tf.concat([w_c[j][1][0][1], w_c[j][1][0][2]], axis=2)], axis=1)
            w_c[j][1][1] = tf.concat([tf.concat([w_c[j+1][1][1], w_c[j][1][1][0]], axis=2),
                                      tf.concat([w_c[j][1][1][1], w_c[j][1][1][2]], axis=2)], axis=1)

        w_0 = tf.concat([tf.concat([w_c[j][0][0], w_c[0][0][0][0]], axis=2),
                         tf.concat([w_c[0][0][0][1], w_c[0][0][0][2]], axis=2)], axis=1)
        w_1 = tf.concat([tf.concat([w_c[j][0][1], w_c[0][0][1][0]], axis=2),
                         tf.concat([w_c[0][0][1][1], w_c[0][0][1][2]], axis=2)], axis=1)
        w_2 = tf.concat([tf.concat([w_c[j][1][0], w_c[0][1][0][0]], axis=2),
                         tf.concat([w_c[0][1][0][1], w_c[0][1][0][2]], axis=2)], axis=1)
        w_3 = tf.concat([tf.concat([w_c[j][1][1], w_c[0][1][1][0]], axis=2),
                         tf.concat([w_c[0][1][1][1], w_c[0][1][1][2]], axis=2)], axis=1)

        w_1234 = tf.concat([tf.concat([w_0, w_1], axis=2), tf.concat([w_2, w_3], axis=2)], axis=1)
        def get_config(self):
            config = super(DTCWT,self).get_config().copy()
            config.update({})
            return config

        return w_1234


class IDTCWT(layers.Layer):
    """
    Inverse Duel Tree Complex Wavelet Transform
    Input: level - tree-level (int)
    """
    def __init__(self, level=1, **kwargs):
        super(IDTCWT, self).__init__(**kwargs)

        if level <= 1:
            level = 1

        self.level = int(level)
        self.conv_type = "SAME"
        self.border_padd = "SYMMETRIC"

        # Faf - First analysis filter - for the first level
        # Fsf - First synthesis filter
        faf, fsf = filters.fs_farras()
        af, sf = filters.duelfilt()

        self.Faf = duel_filter_tf(faf)
        self.Fsf = duel_filter_tf(fsf)
        self.af = duel_filter_tf(af)
        self.sf = duel_filter_tf(sf)

    def build(self, input_shape):
        # repeat last channel if input channel bigger then 1
        if input_shape[-1] > 1:
            self.Faf = tf.repeat(self.Faf, input_shape[-1], axis=-1)
            self.Fsf = tf.repeat(self.Fsf, input_shape[-1], axis=-1)
            self.af = tf.repeat(self.af, input_shape[-1], axis=-1)
            self.sf = tf.repeat(self.sf, input_shape[-1], axis=-1)

    def call(self, inputs, training=None, mask=None):

        # convert one big image into list of tree levels
        w_rec = reconstruct_w_leveln(inputs, self.level)

        height = int(w_rec[0][0][0][0].shape[1]*2)
        width = int(w_rec[0][0][0][0].shape[2]*2)

        # init image to be reconstructed
        y = tf.zeros((height, width, inputs.shape[-1]), dtype=tf.float32)

        w_i = [[[[list() for _ in range(3)], [list() for _ in range(3)]]
                for __ in range(2)] for ___ in range(self.level+1)]

        # first add and subtract (inverse the transform)
        for j in range(self.level):
            for m in range(3):

                w_i[j][0][0][m], w_i[j][1][1][m] = add_sub(w_rec[j][0][0][m], w_rec[j][1][1][m])
                w_i[j][0][1][m], w_i[j][1][0][m] = add_sub(w_rec[j][0][1][m], w_rec[j][1][0][m])

        # synthesis with the First filters to be last in the reconstruction
        for m in range(2):
            for n in range(2):
                lo = w_rec[self.level][m][n]
                for j in [x for x in range(1, self.level)][::-1]:
                    lo = synthesis_filter_bank2d(lo, w_i[j][m][n], self.sf[m][0],
                                                 self.sf[m][1], self.sf[n][0], self.sf[n][1])
                lo = synthesis_filter_bank2d(lo, w_i[0][m][n], self.Fsf[m][0],
                                             self.Fsf[m][1], self.Fsf[n][0], self.Fsf[n][1])
                y = tf.math.add(y, lo)

        # revert the normalization
        y = tf.math.divide(y, 2)
        def get_config(self):
                config = super(IDTCWT,self).get_config().copy()
                config.update({})
                return config

        return y


if __name__ == "__main__":
    pass

In [ ]:
# import tensorflow_wavelets.Layers.DWT as DWT
# import tensorflow_wavelets.Layers.DTCWT as DTCWT
# import tensorflow_wavelets.Layers.DMWT as DMWT
inputs = Input(shape = (128,128,1))
waveTrans1=DTCWT(level=3)(inputs)
# waveTrans1=DMWT.DMWT(wavelet_name='ghm')(inputs)
# waveTrans1=DWTPooling(wavelet="db1")(inputs)
out0 = Cust_Conv(80, (3,3))(waveTrans1)
out1 = Cust_Conv(80, (3,3))(out0)
out2 = Cust_Conv(80, (3,3))(out1)
out3 = Cust_Conv(80, (3,3))(out2)
out3.shape

In [ ]:
waveTrans1.shape

In [ ]:
out4 = Cust_Conv(160, (3,3))(out3)
out5 = Cust_Conv(160, (3,3))(out4)
out6 = Cust_Conv(160, (3,3))(out5)
out7 = Cust_Conv(160, (3,3))(out6)
out7.shape

In [ ]:
out8 = res_layer(160, (3,3))(out7)
out9 = res_layer(160, (3,3))(out8)
out10 = res_layer(160, (3,3))(out9)
out11 = res_layer(160, (3,3))(out10)
out12 = res_layer(160, (3,3))(out11)
out13 = res_layer(160, (3,3))(out12)
out14 = res_layer(160, (3,3))(out13)
out14.shape

In [ ]:
waveadd1 = Add()([out14, out7])
# waveTrans4=IDTCWT(level=3)(waveadd1)
# print(waveTrans4.shape)
out15 = Cust_Conv(160, (3,3))(waveadd1)
out16 = Cust_Conv(160, (3,3))(out15)
out17 = Cust_Conv(160, (3,3))(out16)
out18 = Cust_Conv(160, (3,3))(out17)
out19 = Cust_Conv(80, (3,3))(out18)
out19.shape

In [ ]:
waveadd2 = Add()([out19, out3])
out20 = Cust_Conv(80, (3,3))(out19)
out21 = Cust_Conv(80, (3,3))(out20)
out22 = Cust_Conv(80, (3,3))(out21)
out23 = Cust_Conv(80, (3,3))(out22)
out23.shape

In [ ]:
output = Cust_Conv(filters=1, kernel_size=(1,1))(out23)
output = Add()([output, waveTrans1])
output = IDTCWT(level=3)(output)
output.shape

In [ ]:
import tensorflow as tf
model = tf.keras.Model(inputs = inputs, outputs = output)

In [ ]:
model.compile(
              loss = 'mean_squared_error',
              optimizer = tf.keras.optimizers.Adam(lr=10e-4, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=10e-3)
)
model.summary()

In [ ]:
batch_size=8
checkpoint_path = "D:/SHREYASH_CODE/weights/MWRCNN/checkpoint-{epoch:00d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=9*batch_size)

In [ ]:
model.fit(raw_train_data,clean_train_data,batch_size=batch_size,epochs=25,callbacks=[cp_callback],validation_data=(raw_val_data,clean_val_data))

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
model.load_weights(latest)

In [ ]:
out=model.predict(raw_test_data)
out.shape

In [ ]:
raw_test_data.shape, clean_test_data.shape

In [ ]:
i = random.randint(0, (len(raw_test_data)-1))
i

In [ ]:
plt.imshow(np.reshape(out[i],(128, 128)),cmap='gray')
plt.axis('off')

In [ ]:
plt.imshow(np.reshape(clean_test_data[i],(128, 128)),cmap='gray')
plt.axis('off')

In [ ]:
plt.imshow(np.reshape(raw_test_data[i],(128, 128)),cmap='gray')
plt.axis('off')

In [ ]:
sum1 = 0
sum2 = 0

from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity as ssim
for i in range(len(raw_test_data)):
    outImg=np.reshape(out[i],(128,128))
    sum1 = sum1+peak_signal_noise_ratio(clean_test_data[i],outImg)
    sum2 = sum2+ssim(clean_test_data[i],outImg)

sum1, sum2

In [ ]:
psnr_avg = sum1/len(raw_test_data)
ssim_avg = sum2/len(raw_test_data)

print("Average PSNR: ", psnr_avg)
print("Average SSIM: ", ssim_avg)

In [ ]:
def create_original_dataset(img_folder):
    img_data_array=[]
    path, dirs, files = next(os.walk(img_folder))
    file_count = len(files)
    file_names = sorted(os.listdir(img_folder))
    print(file_count)
    for i in range(file_count):
        imgPath = img_folder+'/'+file_names[i]
        image= cv2.imread( imgPath, cv2.IMREAD_GRAYSCALE)
        image=np.array(image)
        image = image.astype('float32')
        image /= 255
        img_data_array.append(image)
    return img_data_array# extract the image array and class name

In [ ]:
ori_clean = create_original_dataset(clean_test_dir)
ori_clean = np.array(ori_clean)
ori_raw = create_original_dataset(raw_test_dir)
ori_raw = np.array(ori_raw)

In [ ]:
for i in range(len(raw_test_data)):
    print(ori_clean[i].shape, ori_raw[i].shape)

In [ ]:
ori_out = []
for i in range(len(raw_test_data)):
    image = out[i]
    img1 = cv2.resize(image, (900, 450), interpolation = cv2.INTER_AREA)
    img1 = np.array(img1)
    ori_out.append(img1)

ori_out = np.array(ori_out)
ori_out.shape

In [ ]:
i = random.randint(0, (len(raw_test_data)-1))
i

In [ ]:
plt.imshow(np.reshape(ori_out[i],(450, 900)),cmap='gray')
plt.axis('off')

In [ ]:
plt.imshow(np.reshape(ori_clean[i],(450, 900)),cmap='gray')
plt.axis('off')

In [ ]:
plt.imshow(np.reshape(ori_raw[i],(450, 900)),cmap='gray')
plt.axis('off')

In [ ]:
peak_signal_noise_ratio(ori_clean[i], ori_out[i]), ssim(ori_clean[i], ori_out[i])

In [ ]:
sum3 = 0
sum4 = 0

from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity as ssim
for i in range(len(raw_test_data)):
    outImg=ori_out[i]
    sum3 = sum3+peak_signal_noise_ratio(ori_clean[i],outImg)
    sum4 = sum4+ssim(ori_clean[i],outImg)

sum3, sum4

In [ ]:
psnr_avg_ori = sum3/len(raw_test_data)
ssim_avg_ori = sum4/len(raw_test_data)

print("Average PSNR on Original Size: ", psnr_avg_ori)
print("Average SSIM on Original Size: ", ssim_avg_ori)

In [ ]:
out_dir = "D:/SHREYASH_CODE/OUTPUTS/MWRDCNN_DTCWT"

In [ ]:
for i in range(len(ori_out)):
    img = ori_out[i]
    img = Image.fromarray(img)
    img.save(f"{out_dir}/out_{str(i)}.tif")